In [1]:
import h5py
import sys
sys.path.append('/home/amber/multitask_RNA/evaluation/')
import mpra_model
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
from sklearn import model_selection

In [2]:
file = '/home/amber/multitask_RNA/data/lenti_MPRA/K562_onehot_rc.h5'
f = h5py.File(file, 'r')
model = mpra_model.MPRAnn((None,230,4),(None,1))

2023-04-20 14:59:33.084735: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 14:59:37.196741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12507 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:a1:00.0, compute capability: 8.6


In [3]:
x = f['onehot'][()]
y = f['target'][()]

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.1, random_state=42)

In [4]:
earlyStopping_callback = tf.keras.callbacks.EarlyStopping(
            patience=10, restore_best_weights=True
        )
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
                loss="mean_squared_error",
                metrics=["mse", "mae", "mape", "acc"],
                optimizer=optimizer,
            )

In [4]:
result = model.fit(x,f['target'],
        batch_size=128,
        validation_split=0.1,
        epochs=100,
        shuffle=True,
        verbose=2,
        callbacks=[earlyStopping_callback],
    )


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'h5py._hl.dataset.Dataset'>, <class 'h5py._hl.dataset.Dataset'>]